In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytimekr import pytimekr

In [12]:
df = pd.read_csv('./data/30-28_train.csv')
test_df = pd.read_csv('./data/test.csv')
df.head()

,ID,timestamp,item,corporation,location,supply,price,price_t-30,price_t-29,price_t-28,...,price_t + 19,price_t + 20,price_t + 21,price_t + 22,price_t + 23,price_t + 24,price_t + 25,price_t + 26,price_t + 27,price_t + 28
0,TG_A_J_20190131,2019-01-31,TG,A,J,15997.0,2278.0,0.0,0.0,1728.0,...,1581.0,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0
1,TG_A_J_20190201,2019-02-01,TG,A,J,36010.0,2349.0,0.0,1728.0,1408.0,...,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0
2,TG_A_J_20190202,2019-02-02,TG,A,J,8808.0,2286.0,1728.0,1408.0,1250.0,...,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0
3,TG_A_J_20190203,2019-02-03,TG,A,J,0.0,0.0,1408.0,1250.0,0.0,...,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0,0.0
4,TG_A_J_20190204,2019-02-04,TG,A,J,8468.0,1831.0,1250.0,0.0,1474.0,...,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0,0.0,3992.0


In [15]:
holiday = []
for year in range(2019, 2024):
    holiday += pytimekr.holidays(year=year)



def prerpocess(df, holiday=holiday):
    is_train = 'price(원/kg)' in df.columns
    is_holiday = []

    df.timestamp = pd.to_datetime(df.timestamp)
    df['year'] = df.timestamp.dt.year
    df['month'] = df.timestamp.dt.month
    # df['day'] = df.timestamp.dt.day
    df['dow'] = df.timestamp.dt.day_of_week

    for time, dow in zip(df.timestamp, df.dow):
        if time in holiday or dow == 6:
            is_holiday.append(1)
        else:
            is_holiday.append(0)
    df['is_holiday'] = is_holiday

    df = pd.get_dummies(df, columns=['corporation', 'location', 'month'])
    # df = pd.get_dummies(df, columns=['month'])
    
    if is_train:
        df.drop(['ID', 'timestamp', 'supply'], axis=1, inplace=True)
        # df.rename(columns={'price(원/kg)' : 'price'}, inplace=True)
        # df = df.loc[(df['price'] != 0) | (df['dow'] != 6)]
    else:
        df.drop(['ID', 'timestamp'], axis=1, inplace=True)
    
    return df

In [16]:
prerpocess(df).to_csv('./data/train_30-28_preproc_1.csv', index=False)
# prerpocess(test_df).to_csv('./data/test_preproc_lstm1.csv', index=False)
